## Projeto Multimodal Hello World - Iris

Este projeto é uma demonstração prática de como implementar um algoritmo multimodal para classificação de imagens de flores. O algoritmo combina informações de duas fontes diferentes: imagens e dados tabulares.

Primeiro, carregamos um modelo pré-treinado ResNet50V2 e o usamos para extrair características das imagens de flores. As camadas superiores do modelo são congeladas, o que significa que apenas usamos o modelo para extração de características e não o treinamos com nossos dados.

Em seguida, carregamos um conjunto de dados tabulares que contém características adicionais das flores. Usamos um codificador de rótulos para transformar as categorias de flores em números.

Os dados de imagem e tabulares são então divididos em conjuntos de treinamento e teste. O modelo de imagem é usado para extrair características das imagens de treinamento, que são então achatadas e concatenadas com os dados tabulares.

Finalmente, treinamos um modelo de aprendizado de máquina nos dados combinados e usamos o modelo treinado para fazer previsões no conjunto de teste.

Este projeto demonstra como combinar diferentes tipos de dados em um único modelo de aprendizado de máquina e pode ser uma base útil para projetos futuros que envolvem dados multimodais.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Model, Sequential

2024-04-25 19:26:09.532684: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-25 19:26:11.614485: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Carregar o conjunto de dados iris

### Carregar e preparar os dados de imagem:

In [2]:
# Carregar imagens de íris
# Inicializa um gerador de dados de imagem que irá escalar os valores dos pixels para o intervalo [0, 1]
train_datagen = ImageDataGenerator(rescale=1./255)

# Configura o gerador para carregar imagens do diretório especificado, redimensioná-las para 224x224 pixels, carregar em lotes de 32 imagens e usar codificação one-hot para os rótulos de classe
generator = train_datagen.flow_from_directory(
    directory="sample/iris_raw_images",
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

# Extrai os dados do gerador
# Inicializa uma lista para armazenar os dados e um índice de lote
data_list = []
batch_index = 0

# Enquanto o índice do lote for menor ou igual ao índice do lote do gerador, obtém o próximo lote de dados e adiciona as imagens à lista de dados
while batch_index <= generator.batch_index:
    data = next(generator)
    data_list.append(data[0])  # data[0] contém as imagens
    batch_index += 1

# Concatene os dados em um array numpy
# Concatena todos os lotes de imagens em um único array numpy
X_img = np.concatenate(data_list, axis=0)

# Obtem os rótulos de classe das imagens do gerador
y_img = generator.classes

# Separar dados em treino e teste
# Divide os dados de imagem e os rótulos de classe em conjuntos de treinamento e teste
# 80% dos dados serão usados para treinamento e 20% para teste
# 'random_state=42' garante que a divisão seja reproduzível
X_train_img, X_test_img, y_train_img, y_test_img = train_test_split(X_img, y_img, test_size=0.2, random_state=42)

Found 421 images belonging to 3 classes.


In [3]:
# Carrega dataframe com características das flores
# Lê o arquivo CSV que contém as características das imagens das flores e armazena em um dataframe
df = pd.read_csv("sample/iris_raw_images/iris_images.csv")

# Crie um label encoder
# Inicializa um objeto LabelEncoder, que pode transformar rótulos categóricos em números
le = LabelEncoder()

# Ajuste o label encoder e transforme as categorias em números
# Ajusta o LabelEncoder aos rótulos de classe e transforma os rótulos de classe em números
df['classe'] = le.fit_transform(df['classe'])

# Separar dados em treino e teste
# Cria dataframes separados para as características (X_df) e os rótulos de classe (y_df)
X_df = df.drop(["classe", "nome_arquivo"], axis=1)
y_df = df["classe"]

# Divide os dados em conjuntos de treinamento e teste
# 80% dos dados serão usados para treinamento e 20% para teste
# 'random_state=42' garante que a divisão seja reproduzível
X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(X_df, y_df, test_size=0.2, random_state=42)

In [4]:
# Carregar modelo ResNet50V2
# Inicializa o modelo ResNet50V2 com pesos pré-treinados no ImageNet
# 'include_top=False' significa que a última camada (top) do modelo, que é responsável pela classificação, não será incluída
# 'input_shape=(224, 224, 3)' define o formato da entrada para imagens de 224x224 pixels com 3 canais de cor (RGB)
base_model = ResNet50V2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Extrair características das imagens
# Define todas as camadas do modelo base como não treináveis, ou seja, seus pesos não serão atualizados durante o treinamento
for layer in base_model.layers:
    layer.trainable = False

# Cria um novo modelo que tem a mesma entrada que o modelo base e a saída do modelo base como sua saída
# Isso efetivamente cria um modelo que pode ser usado para extrair as características das imagens usando o modelo ResNet50V2
model_img = Model(inputs=base_model.input, outputs=base_model.output)


2024-04-25 19:26:15.090587: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-25 19:26:15.091574: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
# Usa o modelo de imagem para extrair características das imagens de treinamento
# O método 'predict' do modelo é usado para processar as imagens de treinamento através do modelo e obter as características extraídas
features_img = model_img.predict(X_train_img)

11/11 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step


In [6]:
# Criar modelo para características do dataframe
# Inicializa um modelo sequencial
model_df = Sequential() 

# Adiciona a primeira camada densa com 64 neurônios e função de ativação ReLU
model_df.add(Dense(64, activation='relu', input_shape=X_train_df.shape[1:]))

# Adiciona uma camada de dropout para evitar overfitting, descartando 20% dos neurônios
model_df.add(Dropout(0.2))

# Adiciona a segunda camada densa com 32 neurônios e função de ativação ReLU
model_df.add(Dense(32, activation='relu'))

# Adiciona outra camada de dropout para evitar overfitting, descartando 20% dos neurônios
model_df.add(Dropout(0.2))

# Adiciona a camada de saída com um número de neurônios igual ao número de classes únicas em y_train_df e função de ativação softmax
model_df.add(Dense(y_train_df.nunique(), activation='softmax'))

# Compila modelo
# Define a função de perda como entropia cruzada categórica esparsa, o otimizador como Adam e a métrica de avaliação como acurácia
model_df.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


/home/deive/repos/multi_modal_machine_learning/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
# Treina modelo
# O método 'fit' é usado para treinar o modelo 'model_df'
# 'X_train_df' e 'y_train_df' são os dados de treinamento e os rótulos de treinamento, respectivamente
# 'epochs=16' significa que o conjunto de treinamento completo será passado pelo modelo 16 vezes
# 'batch_size=32' define o número de amostras a serem propagadas pela rede de uma vez
# 'validation_split=0.2' reserva 20% dos dados de treinamento para validação
# O histórico de treinamento é armazenado no objeto 'history_df'
history_df = model_df.fit(X_train_df, y_train_df, epochs=16, batch_size=32, validation_split=0.2)

Epoch 1/16
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - accuracy: 0.5571 - loss: 112.1190 - val_accuracy: 0.5588 - val_loss: 28.2549
Epoch 2/16
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4893 - loss: 59.9046 - val_accuracy: 0.6029 - val_loss: 17.1606
Epoch 3/16
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4371 - loss: 39.8852 - val_accuracy: 0.6176 - val_loss: 15.4625
Epoch 4/16
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5174 - loss: 28.6415 - val_accuracy: 0.6324 - val_loss: 14.9826
Epoch 5/16
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4952 - loss: 25.8948 - val_accuracy: 0.6176 - val_loss: 11.2177
Epoch 6/16
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5014 - loss: 26.8512 - val_accuracy: 0.6324 - val_loss: 10.1235
Epoch 7/16
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.4569 - loss: 26.4706 - val_accuracy: 0.6471 - val_loss: 10.8440
Epoch 8/16
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 132ms/step - accuracy: 0.4616 - loss: 66.5756 - val_accuracy: 0.5882 - val_

In [8]:
# Combina características (fusion)
# Redimensiona o array 'features_img' para ter 336 linhas e um número de colunas que será calculado automaticamente (-1)
features_img_flattened = features_img.reshape(336, -1)

# Concatena o array 'features_img_flattened' e o dataframe 'X_train_df' ao longo do eixo 1 (colunas)
# O resultado é um novo array 'features' que contém as colunas de ambos 'features_img_flattened' e 'X_train_df'
features = np.concatenate([features_img_flattened, X_train_df], axis=1)

In [9]:
# Cria modelo final
model_final = Sequential()  # Inicializa um modelo sequencial

# Adiciona a primeira camada densa com 128 neurônios e função de ativação ReLU
model_final.add(Dense(128, activation='relu', input_shape=features.shape[1:]))

# Adiciona uma camada de dropout para evitar overfitting, descartando 30% dos neurônios
model_final.add(Dropout(0.3))

# Adiciona a segunda camada densa com 64 neurônios e função de ativação ReLU
model_final.add(Dense(64, activation='relu'))

# Adiciona outra camada de dropout para evitar overfitting, descartando 20% dos neurônios
model_final.add(Dropout(0.2))

# Adiciona a camada de saída com 3 neurônios (para 3 classes) e função de ativação softmax
model_final.add(Dense(3, activation='softmax'))

# Compila modelo
# Define a função de perda como entropia cruzada categórica esparsa, o otimizador como Adam e a métrica de avaliação como acurácia
model_final.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Treina modelo final
# Treina o modelo usando os dados de treinamento, com 16 épocas, tamanho de lote de 32 e 20% dos dados usados para validação
history_final = model_final.fit(features, y_train_img, epochs=16, batch_size=32, validation_split=0.2)

/home/deive/repos/multi_modal_machine_learning/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/16
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 188ms/step - accuracy: 0.5568 - loss: 9.3776 - val_accuracy: 0.5000 - val_loss: 3.7472
Epoch 2/16
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 151ms/step - accuracy: 0.5390 - loss: 5.1778 - val_accuracy: 0.5882 - val_loss: 3.3714
Epoch 3/16
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 162ms/step - accuracy: 0.6170 - loss: 4.6311 - val_accuracy: 0.4706 - val_loss: 2.5597
Epoch 4/16
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 154ms/step - accuracy: 0.7031 - loss: 2.9114 - val_accuracy: 0.5882 - val_loss: 2.8974
Epoch 5/16
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 171ms/step - accuracy: 0.7805 - loss: 1.4437 - val_accuracy: 0.4853 - val_loss: 2.6813
Epoch 6/16
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 190ms/step - accuracy: 0.7770 - loss: 1.3047 - val_accuracy: 0.6471 - val_loss: 2.6569
Epoch 7/16
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 185ms/step - accuracy: 0.8260 - loss: 2.1822 - val_accuracy: 0.6176 - val_loss: 2.5201
Epoch 8/16
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 183ms/step - accuracy: 0.8238 - loss: 0.7894 - val_accuracy: 0.6029 - val_loss:

In [10]:
# Usa o modelo de imagem para extrair características das imagens de teste
# O método 'predict' do modelo é usado para processar as imagens de teste através do modelo e obter as características extraídas
features_img_test = model_img.predict(X_test_img)

# Redimensiona as características da imagem para terem uma dimensão de (número de imagens, -1)
# O -1 significa que o tamanho da segunda dimensão é calculado automaticamente com base no tamanho da matriz original
features_img_flattened_test = features_img_test.reshape(len(y_test_img), -1)

# Concatena as características da imagem e as características do dataframe para criar um conjunto de características completo para teste
# 'axis=1' significa que a concatenação é feita ao longo do eixo das colunas (ou seja, as características do dataframe são adicionadas como colunas adicionais às características da imagem)
features_test = np.concatenate([features_img_flattened_test, X_test_df], axis=1)

3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step


In [11]:
# Usa o modelo final para fazer previsões no conjunto de teste
# O método 'predict' do modelo é usado para processar as características de teste através do modelo e obter as previsões
# As previsões são armazenadas na variável 'prevision'
prevision = model_final.predict(features_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


In [12]:
# Converte probabilidade para labels da classe
predicted_classes = np.argmax(prevision, axis=1)

In [13]:
# Calcula acurácia
accuracy = accuracy_score(y_test_img, predicted_classes)

print(f'Acurácia do modelo: {accuracy * 100:.2f}%')

Acurácia do modelo: 56.47%
